In [42]:
import itertools
import logging
import re

import numpy as np
import pandas as pd
import toml
import s3fs

from goes2go import config
from goes2go.data import goes_latest, goes_nearesttime, goes_timerange, _goes_file_df

log = logging.getLogger(__name__)

# Connect to AWS public buckets
fs = s3fs.S3FileSystem(anon=True)

In [11]:
product_table = pd.read_csv("./product_table.txt", skiprows=2, names=['product', 'description'], index_col=0)
product_table.index = product_table.index.str.strip()
product_table['description'] = product_table.description.str.strip()
product_table

,description
product,
ABI-L1b-RadF,Advanced Baseline Imager Level 1b Full Disk
ABI-L1b-RadC,Advanced Baseline Imager Level 1b CONUS
ABI-L1b-RadM,Advanced Baseline Imager Level 1b Mesoscale
ABI-L2-ACHAC,Advanced Baseline Imager Level 2 Cloud Top Hei...
ABI-L2-ACHAF,Advanced Baseline Imager Level 2 Cloud Top Hei...
...,...
SUVI-L1b-Fe131,Solar Ultraviolet Imager Level 1b Extreme Ultr...
SUVI-L1b-Fe171,Solar Ultraviolet Imager Level 1b Extreme Ultr...
SUVI-L1b-Fe195,Solar Ultraviolet Imager Level 1b Extreme Ultr...


In [12]:
# Assume goes17 and goes18 have same products as goes16
_product = {i.split("/")[-1] for i in fs.ls(f"noaa-goes16")}
_product = set(filter(lambda x: x.split('.')[-1] not in ['pdf', 'html'], _product))
_product

# you can be unspecific and request any mesoscale domain (M), 
# or by number (M1, M2)
_domains = {"F", "C", "M", "M1", "M2"} 

In [19]:
product_table.loc['ABI-L1b-RadF'].description

'Advanced Baseline Imager Level 1b Full Disk'

In [43]:
class GOES:
    def __init__(
        self,
        satellite=config["timerange"].get("satellite"),
        product=config["timerange"].get("product"),
        domain=config["timerange"].get("domain"),
    ):
        """

        Parameters
        ----------
        satellite : {16, 17, 18}
            The satellite number. May also use the following aliases
            {'G16', "G17", "EAST", "WEST"}
        product : str
            The product to aquire.
            - GLM = alias for geostationary lighting mapper
            - ABI = alias for ABI multi-channel cloud moisture imagery
        domain : {None, 'F', 'C', "M", "M1", "M2"}
            Only needed for ABI products.
            - F = Full Disk
            - C = CONUS
            - M = Mesoscale sector (both)
            - M1 = Mesocale sector 1
            - M2 = Mesocale sector 2
        """
        self.satellite = satellite
        self.product = product
        self.domain = domain

        self._check_satellite()
        self._check_product()

    def _check_satellite(self):
        if isinstance(self.satellite, int):
            self.satellite = f"noaa-goes{self.satellite}"
        elif isinstance(self.satellite, str):
            if self.satellite.upper() == "EAST":
                self.satellite = "noaa-goes16"
            elif self.satellite.upper() == "WEST":
                self.satellite = "noaa-goes17"
            else:
                # look for the satellite number in the string (i.e.g, 'G16', 'goes16')
                self.satellite = re.sub("[^0-9]", "", self.satellite)
                self.satellite = f"noaa-goes{self.satellite}"
        else:
            raise ValueError(
                f"Could not figure out what satellite you want from `{self.satellite}`"
            )

    def _check_product(self):
        if self.product == "GLM":
            # Alias for geostationary lighting mapper
            self.product = "GLM-L2-LCFA"
        elif self.product == "ABI":
            # Alias for multi-channel cloud moisture imagery
            if self.domain is None:
                self.product = "ABI-L2-MCMIP" + "C"
            else:
                self.product = "ABI-L2-MCMIP" + re.sub("[0-9]", "", self.domain)
        elif self.product.startswith("ABI"):
            if self.domain is None:
                if self.product[-1] in _domains:
                    self.domain = self.product[-1]

            elif self.domain is not None:
                if self.domain in _domains:
                    self.product = self.product + re.sub("[0-9]", "", self.domain)
                    if self.product not in _product:
                        raise ValueError(
                            f"{self.product} not a valid product product. Must one of {_domains}"
                        )
                else:
                    raise ValueError(
                        f"domain for ABI products must be None or one of {_domains}"
                    )
        else:
            if self.domain is not None:
                log.warning("domain argument is ignored for non-ABI products")

        if self.product in _product:
            self.description = product_table.loc[self.product].description
        else:
            raise ValueError(f"{self.product} is not an available product.")

    def __repr__(self):
        msg = [
            f"--- This GOES Object ---",
            f"{self.satellite=}",
            f"{self.product=}",
            f"{self.domain=}",
            f"{self.description=}",
        ]
        return "\n".join(msg)

    def latest(self, **kwargs):
        return goes_latest(
            satellite=self.satellite, product=self.product, domain=self.domain, **kwargs
        )

    def nearest(self, attime, within, **kwargs):
        return goes_latest(
            attime,
            within,
            satellite=self.satellite,
            product=self.product,
            domain=self.domain,
            **kwargs,
        )

    def goes_timerange(self, start=None, end=None, recent=None, **kwargs):
        return goes_timerange(
            start,
            end,
            recent,
            satellite=self.satellite,
            product=self.product,
            domain=self.domain,
            **kwargs,
        )

    def df(self, start, end, bands=None, refresh=True):
        return _goes_file_df(self.satellite, self.product, start=start, end=end, bands=bands, refresh=refresh)

In [48]:
G = GOES(domain='M')
G

--- This GOES Object ---
self.satellite='noaa-goes16'
self.product='ABI-L2-MCMIPM'
self.domain='M'
self.description='Advanced Baseline Imager Level 2 Cloud and Moisture Imagery Mesoscale'

In [49]:
G.df('2021-01-01', '2021-01-01 01')

,file,product_mode,satellite,start,end,creation
0,noaa-goes16/ABI-L2-MCMIPM/2021/001/00/OR_ABI-L...,ABI-L2-MCMIPM1-M6,G16,2021-01-01 00:00:28.400,2021-01-01 00:00:35.300,2021-01-01 00:00:44.000
1,noaa-goes16/ABI-L2-MCMIPM/2021/001/00/OR_ABI-L...,ABI-L2-MCMIPM1-M6,G16,2021-01-01 00:01:25.500,2021-01-01 00:01:31.800,2021-01-01 00:01:39.900
2,noaa-goes16/ABI-L2-MCMIPM/2021/001/00/OR_ABI-L...,ABI-L2-MCMIPM1-M6,G16,2021-01-01 00:02:25.500,2021-01-01 00:02:31.200,2021-01-01 00:02:39.500
3,noaa-goes16/ABI-L2-MCMIPM/2021/001/00/OR_ABI-L...,ABI-L2-MCMIPM1-M6,G16,2021-01-01 00:03:25.500,2021-01-01 00:03:32.400,2021-01-01 00:03:40.000
4,noaa-goes16/ABI-L2-MCMIPM/2021/001/00/OR_ABI-L...,ABI-L2-MCMIPM1-M6,G16,2021-01-01 00:04:25.500,2021-01-01 00:04:31.800,2021-01-01 00:04:40.100
...,...,...,...,...,...,...
114,noaa-goes16/ABI-L2-MCMIPM/2021/001/00/OR_ABI-L...,ABI-L2-MCMIPM2-M6,G16,2021-01-01 00:54:55.500,2021-01-01 00:55:01.200,2021-01-01 00:55:09.800
115,noaa-goes16/ABI-L2-MCMIPM/2021/001/00/OR_ABI-L...,ABI-L2-MCMIPM2-M6,G16,2021-01-01 00:55:55.500,2021-01-01 00:56:02.400,2021-01-01 00:56:08.800
116,noaa-goes16/ABI-L2-MCMIPM/2021/001/00/OR_ABI-L...,ABI-L2-MCMIPM2-M6,G16,2021-01-01 00:56:55.500,2021-01-01 00:57:01.200,2021-01-01 00:57:09.300
117,noaa-goes16/ABI-L2-MCMIPM/2021/001/00/OR_ABI-L...,ABI-L2-MCMIPM2-M6,G16,2021-01-01 00:57:55.500,2021-01-01 00:58:02.400,2021-01-01 00:58:09.700


In [41]:
G.goes_timerange(recent='1H', return_as='filelist')

 _______________________________
 | Satellite: noaa-goes16      |
 |   Product: ABI-L2-MCMIPC    |
 |    Domain: C                |
📦 Finished downloading [11] files to [C:\Users\blayl_depgywe\data\noaa-goes16\ABI-L2-MCMIPC].


,file,product_mode,satellite,start,end,creation
0,noaa-goes16/ABI-L2-MCMIPC/2022/127/17/OR_ABI-L...,ABI-L2-MCMIPC-M6,G16,2022-05-07 17:06:17.200,2022-05-07 17:08:54.500,2022-05-07 17:09:04.900
1,noaa-goes16/ABI-L2-MCMIPC/2022/127/17/OR_ABI-L...,ABI-L2-MCMIPC-M6,G16,2022-05-07 17:11:17.200,2022-05-07 17:13:54.500,2022-05-07 17:14:04.300
2,noaa-goes16/ABI-L2-MCMIPC/2022/127/17/OR_ABI-L...,ABI-L2-MCMIPC-M6,G16,2022-05-07 17:16:17.200,2022-05-07 17:18:54.500,2022-05-07 17:19:04.300
3,noaa-goes16/ABI-L2-MCMIPC/2022/127/17/OR_ABI-L...,ABI-L2-MCMIPC-M6,G16,2022-05-07 17:21:17.200,2022-05-07 17:23:54.500,2022-05-07 17:24:05.000
4,noaa-goes16/ABI-L2-MCMIPC/2022/127/17/OR_ABI-L...,ABI-L2-MCMIPC-M6,G16,2022-05-07 17:26:17.200,2022-05-07 17:28:54.500,2022-05-07 17:29:04.500
5,noaa-goes16/ABI-L2-MCMIPC/2022/127/17/OR_ABI-L...,ABI-L2-MCMIPC-M6,G16,2022-05-07 17:31:17.200,2022-05-07 17:33:55.000,2022-05-07 17:34:05.000
6,noaa-goes16/ABI-L2-MCMIPC/2022/127/17/OR_ABI-L...,ABI-L2-MCMIPC-M6,G16,2022-05-07 17:36:17.200,2022-05-07 17:38:55.000,2022-05-07 17:39:05.200
7,noaa-goes16/ABI-L2-MCMIPC/2022/127/17/OR_ABI-L...,ABI-L2-MCMIPC-M6,G16,2022-05-07 17:41:17.200,2022-05-07 17:43:54.500,2022-05-07 17:44:04.500
8,noaa-goes16/ABI-L2-MCMIPC/2022/127/17/OR_ABI-L...,ABI-L2-MCMIPC-M6,G16,2022-05-07 17:46:17.200,2022-05-07 17:48:54.500,2022-05-07 17:49:04.500
9,noaa-goes16/ABI-L2-MCMIPC/2022/127/17/OR_ABI-L...,ABI-L2-MCMIPC-M6,G16,2022-05-07 17:51:17.200,2022-05-07 17:53:55.000,2022-05-07 17:54:04.600
